In [1]:
import os
import tensorflow as tf

print(tf.version.VERSION)

2.3.0


In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [5]:
def create_model():
  model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])

  return model

In [6]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [8]:
#!ls {checkpoint_dir}
!dir {checkpoint_dir}

 Volume in drive D has no label.
 Volume Serial Number is 548E-A4BA

 Directory of D:\Sandbox\Github\TF_786\1. Introduction to TensorFlow\AZ



File Not Found


In [9]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*batch_size)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 00005: saving model to training_2\cp-0005.ckpt

Epoch 00010: saving model to training_2\cp-0010.ckpt

Epoch 00015: saving model to training_2\cp-0015.ckpt

Epoch 00020: saving model to training_2\cp-0020.ckpt

Epoch 00025: saving model to training_2\cp-0025.ckpt

Epoch 00030: saving model to training_2\cp-0030.ckpt

Epoch 00035: saving model to training_2\cp-0035.ckpt

Epoch 00040: saving model to training_2\cp-0040.ckpt

Epoch 00045: saving model to training_2\cp-0045.ckpt

Epoch 00050: saving model to training_2\cp-0050.ckpt


Now, look at the resulting checkpoints and choose the latest one:

In [10]:
#!ls {checkpoint_dir}
!dir {checkpoint_dir}

 Volume in drive D has no label.
 Volume Serial Number is 548E-A4BA

 Directory of D:\Sandbox\Github\TF_786\1. Introduction to TensorFlow\AZ\training_2

03/21/2021  04:50 PM    <DIR>          .
03/21/2021  04:50 PM    <DIR>          ..
03/21/2021  04:50 PM                81 checkpoint
03/21/2021  04:50 PM         1,628,726 cp-0000.ckpt.data-00000-of-00001
03/21/2021  04:50 PM               402 cp-0000.ckpt.index
03/21/2021  04:50 PM         4,886,685 cp-0005.ckpt.data-00000-of-00001
03/21/2021  04:50 PM             1,222 cp-0005.ckpt.index
03/21/2021  04:50 PM         4,886,685 cp-0010.ckpt.data-00000-of-00001
03/21/2021  04:50 PM             1,222 cp-0010.ckpt.index
03/21/2021  04:50 PM         4,886,685 cp-0015.ckpt.data-00000-of-00001
03/21/2021  04:50 PM             1,222 cp-0015.ckpt.index
03/21/2021  04:50 PM         4,886,685 cp-0020.ckpt.data-00000-of-00001
03/21/2021  04:50 PM             1,222 cp-0020.ckpt.index
03/21/2021  04:50 PM         4,886,685 cp-0025.ckpt.data-00000-o

In [11]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0050.ckpt'

Note: the default TensorFlow format only saves the 5 most recent checkpoints.

To test, reset the model and load the latest checkpoint:

In [12]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4633 - sparse_categorical_accuracy: 0.8820
Restored model, accuracy: 88.20%


In [13]:
# Manually Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4633 - sparse_categorical_accuracy: 0.8820
Restored model, accuracy: 88.20%
